In [ ]:
!pip -q install langchain openai tiktoken PyPDF2 faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.0/116.0 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

# Chat & Query your PDF files

In [ ]:
import os

In [ ]:
from  secret_key import openapi_key

In [ ]:
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
!pip show langchain

Name: langchain
Version: 0.1.16
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, jsonpatch, langchain-community, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


## The Game plan


<img src="https://dl.dropboxusercontent.com/s/gxij5593tyzrvsg/Screenshot%202023-04-26%20at%203.06.50%20PM.png" alt="vectorstore">


<img src="https://dl.dropboxusercontent.com/s/v1yfuem0i60bd88/Screenshot%202023-04-26%20at%203.52.12%20PM.png" alt="retreiver chain">


In [ ]:
# Download the PDF Reid Hoffman book with GPT-4 from his free download link
!wget -q https://www.impromptubook.com/wp-content/uploads/2023/03/impromptu-rh.pdf

### Basic Chat PDF


In [ ]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

## Reading in the PDF


In [ ]:
# location of the pdf file/files.
doc_reader = PdfReader('/content/impromptu-rh.pdf')

In [ ]:
len(doc_reader.pages)

247

In [ ]:
# read data from the file and put them into a variable called raw_text
raw_text = ''
for i, page in enumerate(doc_reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

In [ ]:
len(raw_text)

371090

In [ ]:
print(raw_text[:10000])

Impromptu
Amplifying Our Humanity 
Through AI
By Reid Hoffman  
with GPT-4Impromptu: AmplIfyIng our HumAnIty tHrougH AI  
by Reid Hoffman with GPT-4
ISBNs:  979-8-9878319-1-5 Trade Paperback  
979-8-9878319-2-2 Hardcover 
979-8-9878319-0-8 Ebook
Copyright 2023 Dallepedia LLC
Published by Dallepedia LLC. All rights reserved. No portion 
of this work may be reproduced in any form, with the limited 
exception of brief quotations in editorial reviews, without express 
written permission from the publisher.“The analytical engine weaves algebraic patterns, like 
the loom weaves flowers and leaves. Artificial intel-
ligence can embroider this fabric of logic with the 
colours of imagination and creativity.”
—ADA LOVELACE, AS IMAGINED BY GPT-4
“Artificial intelligence is not a separate entity from us, 
but a reflection of our own mind. By cultivating it with 
skillful means and ethical values, we can enhance our 
own enlightenment and benefit all beings.”
—THE BUDDHA, AS IMAGINED BY GPT-4vCONT

### Text Splitter

This takes the text and splits it into chunks. The chunk size is characters not tokens

In [ ]:
# Splitting up the text into smaller chunks for indexing
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200, #striding over the text
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [ ]:
len(texts)

466

In [ ]:
print(texts[0])

Impromptu
Amplifying Our Humanity 
Through AI
By Reid Hoffman  
with GPT-4Impromptu: AmplIfyIng our HumAnIty tHrougH AI  
by Reid Hoffman with GPT-4
ISBNs:  979-8-9878319-1-5 Trade Paperback  
979-8-9878319-2-2 Hardcover 
979-8-9878319-0-8 Ebook
Copyright 2023 Dallepedia LLC
Published by Dallepedia LLC. All rights reserved. No portion 
of this work may be reproduced in any form, with the limited 
exception of brief quotations in editorial reviews, without express 
written permission from the publisher.“The analytical engine weaves algebraic patterns, like 
the loom weaves flowers and leaves. Artificial intel-
ligence can embroider this fabric of logic with the 
colours of imagination and creativity.”
—ADA LOVELACE, AS IMAGINED BY GPT-4
“Artificial intelligence is not a separate entity from us, 
but a reflection of our own mind. By cultivating it with 
skillful means and ethical values, we can enhance our 
own enlightenment and benefit all beings.”


In [ ]:
print(texts[1])

but a reflection of our own mind. By cultivating it with 
skillful means and ethical values, we can enhance our 
own enlightenment and benefit all beings.”
—THE BUDDHA, AS IMAGINED BY GPT-4vCONTENTS
Introduction: Moments of Enlightenment   . . . . . . . 1
1. Education   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 25
2. Creativity . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 49
3. Justice   . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 71
4. Journalism   . . . . . . . . . . . . . . . . . . . . . . . . . . . . 83
5. Social Media   . . . . . . . . . . . . . . . . . . . . . . . . . . 105
6. Transformation of Work   . . . . . . . . . . . . . . . . 119
7. GPT-4 In My Own Work  . . . . . . . . . . . . . . . . 143
8. When AI Makes Things Up  
(“Hallucinations”)   . . . . . . . . . . . . . . . . . . . . . 165
9. Public Intellectuals . . . . . . . . . . . . . . . . . . . . . 179
10. Homo Techne  . . . . . . . . . . . . . . . . . . . . . . . . . 203


## Making the embeddings

In [ ]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
docsearch = FAISS.from_texts(texts, embeddings)

In [ ]:
docsearch

In [ ]:
docsearch.embedding_function

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x7aa245c5d8a0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7aa245a88a90>, model='text-embedding-ada-002', deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key='sk-proj-ttOfVTdStspbikn3lWzmT3BlbkFJJo2BHIxnfQYcRat8cnfJ', openai_organization=None, allowed_special=set(), disallowed_special='all', chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None)

In [ ]:
query = "how does GPT-4 change social media?"
docs = docsearch.similarity_search(query)

In [ ]:
len(docs)

4

In [ ]:
docs

[Document(page_content='organizational psychologist whose books have titles such as \nWhy Do So Many Incompetent Men Become Leaders?\nMintz agreed with Chamorro-Premuzic that humans could \nthrive alongside AI by: (1) specializing in asking the best ques-\ntions, (2) learning insights or skills that are not available in the \n“training data” used by the deep learning networks, and (3) \nturning insights into actions.\nReid: GPT-4, do you agree that these will remain dis-\ntinctive human capabilities even as your own capabili-\nties grow?\nGPT-4:  I agree. Asking the best questions requires curi-\nosity, creativity, and critical thinking, which are not easily \nreplicated or replaced by AI. Learning insights or skills \nthat are not in the training data requires exploration, \nexperimentation, and discovery, which are not always 28Impromptu: Amplifying Our Humanity Through AI\npredictable or optimal for AI. Turning insights into \nactions requires judgment, communication, and leader -')

In [ ]:
query = "who is Rinku singh"
docs = docsearch.similarity_search(query)

In [ ]:
docs

[Document(page_content='organizational psychologist whose books have titles such as \nWhy Do So Many Incompetent Men Become Leaders?\nMintz agreed with Chamorro-Premuzic that humans could \nthrive alongside AI by: (1) specializing in asking the best ques-\ntions, (2) learning insights or skills that are not available in the \n“training data” used by the deep learning networks, and (3) \nturning insights into actions.\nReid: GPT-4, do you agree that these will remain dis-\ntinctive human capabilities even as your own capabili-\nties grow?\nGPT-4:  I agree. Asking the best questions requires curi-\nosity, creativity, and critical thinking, which are not easily \nreplicated or replaced by AI. Learning insights or skills \nthat are not in the training data requires exploration, \nexperimentation, and discovery, which are not always 28Impromptu: Amplifying Our Humanity Through AI\npredictable or optimal for AI. Turning insights into \nactions requires judgment, communication, and leader -')

## Plain QA Chain

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [ ]:
chain = load_qa_chain(OpenAI(),
                      chain_type="stuff") # we are going to stuff all the docs in at once

In [ ]:
chain.llm_chain.prompt

PromptTemplate(input_variables=['context', 'question'], template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:")

In [ ]:
# check the prompt
chain.llm_chain.prompt.template

"Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:"

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:

In [ ]:
query = "who are the authors of the book?"
docs = docsearch.similarity_search(query,k = 3)
chain.run(input_documents=docs, question=query)

' Reid Hoffman, Sam Altman, and the team at OpenAI'

In [ ]:
docs = docsearch.similarity_search(query, k = 3)

In [ ]:
print(docs[2].page_content)

di Cesare. I have read your book, Philosophy and the 
Public, with great interest and admiration. You argue 
that philosophy has a vital role to play in addressing the 
urgent challenges of our time, such as democracy, vio -
lence, ecology, and human rights. You also criticize the 
academic specialization and detachment of much con-
temporary philosophy, and call for a more engaged and 
dialogical approach. Can you tell us more about what 
motivated you to write this book, and what you hope to 
achieve with it?
di Cesare: Thank you, Mr. Ishiguro, for your kind words 
and for this opportunity to discuss my book with you. I 
wrote this book because I believe that philosophy is not 
a luxury or a hobby, but a necessity and a responsibility. 
Philosophy is not only a way of thinking, but also a way 
of living, a way of being in the world, a way of question-
ing and challenging the status quo, a way of imagining 
and creating alternatives. Philosophy is not a closed


In [ ]:
query = "who is the author of the book?"
query_02 = "has it rained this week?"
docs = docsearch.similarity_search(query_02)
chain.run(input_documents=docs, question=query)

'\n\nThe author of the book is not mentioned in the given context. Therefore, I do not know the answer.'

In [105]:
docs

[Document(page_content='on all of these issues in a deeper and more nuanced manner \nthan is possible here.JOURNALISM\nFor the American journalism industry, it’s been twenty \nyears of mostly bad news. With the rise of the internet, compe -\ntition for ad dollars from non-news players has destroyed the \nindustry’s traditional business models—even as participation \nfrom a public that, finally empowered to talk back, has slowly \neroded journalism’s authority.\nNewspaper publishers—which have traditionally done the \nheavy lifting of holding power accountable and informing the \npublic about current affairs—have suffered the worst of it. \nAccording to the Pew Research Center , more than 2,200 local \nU.S. papers have closed since 2005, and over 40,000 news -\nroom employees have lost their jobs.\nMeanwhile, we keep producing more and more information. \nLike, a lot of it:\nReid: Can you quantify how much information the world \nproduces each day in 2022? Also, can you provide any'),
 

In [106]:
query = "who is the book authored by?"
docs = docsearch.similarity_search(query,k=4)
chain.run(input_documents=docs, question=query)

' The book is authored by Reid Hoffman.'

In [107]:
docs

[Document(page_content='di Cesare. I have read your book, Philosophy and the \nPublic, with great interest and admiration. You argue \nthat philosophy has a vital role to play in addressing the \nurgent challenges of our time, such as democracy, vio -\nlence, ecology, and human rights. You also criticize the \nacademic specialization and detachment of much con-\ntemporary philosophy, and call for a more engaged and \ndialogical approach. Can you tell us more about what \nmotivated you to write this book, and what you hope to \nachieve with it?\ndi Cesare: Thank you, Mr. Ishiguro, for your kind words \nand for this opportunity to discuss my book with you. I \nwrote this book because I believe that philosophy is not \na luxury or a hobby, but a necessity and a responsibility. \nPhilosophy is not only a way of thinking, but also a way \nof living, a way of being in the world, a way of question-\ning and challenging the status quo, a way of imagining \nand creating alternatives. Philosophy

### QA Chain with mapreduce

In [108]:
chain = load_qa_chain(OpenAI(),
                      chain_type="stuff") # we are going to stuff all the docs in at once

In [112]:
query = "who is the book authored by?"
docs = docsearch.similarity_search(query,k=5)
chain.run(input_documents=docs, question=query)

' The book is authored by Reid Hoffman and GPT-4.'

In [ ]:
chain = load_qa_chain(OpenAI(),
                      chain_type="map_rerank",
                      return_intermediate_steps=True
                      )

query = "who are openai?"
docs = docsearch.similarity_search(query,k=5)
results = chain({"input_documents": docs, "question": query}, return_only_outputs=True)
results

In [ ]:
results['output_text']

' OpenAI is a research organization that develops and shares artificial intelligence tools for the benefit of humanity.'

In [ ]:
results['intermediate_steps']

[{'answer': ' OpenAI is an organization that released text-to-image generation tool DALL-E 2 and ChatGPT in April 2022 and are giving millions of users hands-on access to these AI tools.',
  'score': '80'},
 {'answer': ' OpenAI is a research laboratory whose founding goal is to develop technologies that put the power of AI directly into the hands of millions of people.',
  'score': '80'},
 {'answer': ' OpenAI is a research organization that develops and shares artificial intelligence tools for the benefit of humanity.',
  'score': '100'},
 {'answer': ' OpenAI is a technology company focused on using artificial intelligence to solve real-world problems.',
  'score': '80'},
 {'answer': ' OpenAI is a company co-founded by Sam Altman that is developing AI technologies and allowing individuals to participate in the development process.',
  'score': '90'},
 {'answer': ' OpenAI is a research laboratory that focuses on artificial intelligence technologies.',
  'score': '80'},
 {'answer': ' Ope

In [116]:
# check the prompt
print(chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

In addition to giving an answer, also return a score of how fully it answered the user's question. This should be in the following format:

Question: [question here]
Helpful Answer: [answer here]
Score: [score between 0 and 100]

How to determine the score:
- Higher is a better answer
- Better responds fully to the asked question, with sufficient level of detail
- If you do not know the answer based on the context, that should be a score of 0
- Don't be overconfident!

Example #1

Context:
---------
Apples are red
---------
Question: what color are apples?
Helpful Answer: red
Score: 100

Example #2

Context:
---------
it was night and the witness forgot his glasses. he was not sure if it was a sports car or an suv
---------
Question: what type was the car?
Helpful Answer: a sports car or an suv
Score: 60

Example #3

Context

## RetrievalQA
RetrievalQA chain uses load_qa_chain and combines it with the a retriever (in our case the FAISS index)

In [117]:
from langchain.chains import RetrievalQA

# set up FAISS as a generic retriever
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":4})

# create the chain to answer questions
rqa = RetrievalQA.from_chain_type(llm=OpenAI(),
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [118]:
rqa("What is OpenAI?")

{'query': 'What is OpenAI?',
 'result': ' OpenAI is an organization that was founded in 2015 with the goal of developing technologies that put the power of AI directly into the hands of millions of people. It has since released various AI tools and programs, including DALL-E 2 and ChatGPT, and aims to achieve good outcomes for society in the long term. It was also created to protect society from potential negative impacts of AI.',
 'source_documents': [Document(page_content='ing to their own lives however they best saw fit.\nEspecially in the realm of work, I realized, AI deployed in this \nway could give individuals incredibly versatile new tools to \napply to their careers, professional development, and economic \nautonomy. So when I had a chance to become one of OpenAI’s \ninitial funders in 2015, I took it. The vision of AI that it was \nplanning to pursue felt like a natural extension of the goals that \nhad inspired me to co-found LinkedIn in 2002.\nWhen OpenAI released its text-

In [121]:
query = "What does gpt-4 mean for creativity?"
rqa(query)['result']

'\nGPT-4 has the potential to greatly impact creativity by assisting human creators with tasks such as brainstorming, editing, feedback, and translation. It can also generate various forms of media such as images, music, and video. This can benefit artists, consumers, and society at large. However, there are also concerns about the impact of GPT-4 on the originality and authenticity of creative work.'

In [122]:
query = "what have the last 20 years been like for American journalism?"
rqa(query)['result']

' The last 20 years for American journalism have been characterized by mostly bad news, including a decline in traditional business models due to competition from non-news players, erosion of authority due to empowered public participation, and job losses in the industry. There has also been a significant increase in information production and the use of large language models in the journalism industry.'

In [ ]:
query = "how can journalists use GPT-4??"
rqa(query)['result']

' Journalists can use GPT-4 to augment and enhance their work and make them more productive. They can use it to evaluate sources, data and information, identify gaps, biases and errors, and generate original insights, angles, and questions. However, AI-generated headlines and captions should still be reviewed and approved by human editors to ensure that they are factually accurate, ethically sound, and in line with the tone and values of the news organization.'

In [ ]:
query = "How is GPT-4 different from other models?"
rqa(query)['result']

' GPT-4 arranges vast, unstructured arrays of human knowledge and expression into a more connected and interoperable network, enabling a new kind of highly contextualized search. It can also be used to generate images, data analysis, code writing, 3D models, lighting effects, audio editing, and more.'

In [ ]:
query = "What is beagle Bard?"
rqa(query)['result']

' Beagle Bard is not mentioned in the context given.'